In [ ]:
import spikeinterface.widgets as sw
from AODR_session_sorters import OpenEphysSessionSorter as OES
from pyramid import cli
import pandas as pd
import sys, os
%matplotlib ipympl

Below, we define the relevant paths. You should change these to match the corresponding paths on your machine. If you encounter issues while loading/sorting the file, chances are that your computer is having a hard time accessing the files from cloud storage. If that is the case, you need to transfer your files to a local directory, and also save the files to a local directory.

In [ ]:
# Experiment directory top level - should contain the subfolders like ecodes, python, matlab.
expDir = "C:/Users/lt711/Documents/GitHub/Lab_Pipelines/experiments/aodr"
sessDir = "MrM_2025-10-31_11-31-10" #"MrM_2025-09-26-Trimmed"
os.chdir(expDir) # regular python files will run from the first open folder in your tree, but notebooks will work from wherever the notebook is stored.
# Directory where the raw files are stored to be converted/sorted
dataSearchPath = "C:/Users/lt711/Box/GoldLab/Data/Physiology/AODR/dlPFC/MrM/Raw/Neuronal/Tetrode/"
# Where the rules for ecodes are stored
pyramidSearchPath = expDir+"/ecodes"
# Conversion specifications
convertSpecs = expDir+"/AODR_experiment_tetrode.yaml"
# Base directory to save the output files from pyramid (hdf5 files)
baseSaveDir = "C:/NeuronalData/Converted/"
# The Open Ephys experiment name
currentFile = "experiment1.nwb"
# Full directory to save the output files from pyramid (hdf5 files)
trialFileOutputName = baseSaveDir+sessDir+".hdf5"
# Directory to save the output files from sorting
sorted_out = 'C:/NeuronalData/Sorted/'+sessDir+"/"
 
sys.path.append(expDir+"/python") # to make sure pyramid can access the custom collectors/enhancers/functions?

Below, we run the sorter/analyzer. With ~4 channels of neural data and a 90 min recording, this should take between 10-20 minutes. The step_names variable defines the sequential processing steps that are defined as methods in AODR_session_sorters OpenEphysSessionSorter class. If you use the step "open_sigui", a gui window will open that allows to interact with the sorting results. Close the gui to continue to the next step and save the output.

In [ ]:
sorter = OES(session_dir=dataSearchPath+sessDir+"/", channel_names=[1,2,3,4],
             out_folder=sorted_out,
             stream_name='acquisition_board',
             overwrite_timestamps=True,             # This will overwrite the first sample time in the phy params.py file since the first sample number has non-zero time
             sorter_name='single_channel',
             step_names=[
                 'clean_tree',                      # This removes everything from the output folder!!!
                 'read_data',                       # Has to be included to do anything else
                 'set_tetrode',
                 'bandpass',
                 'single_ch_sorter_and_analyzer',     # Runs one spike sorter which is an optional parameter (default spikecyrcus2) and one analyzer
                 'open_sigui'
             ])

In [ ]:
sorter.export_to_phy()

Now, run Pyramid on the corresponding output to create a TrialFile. Make sure that the phy_reader.params_file points to the appropriate directory based on those you have defined above.

You need to ensure that the phy reader has the offset param!!!!!!
times = (self.spikes_times[self.current_row:until_row] / self.sample_rate) + self.offset

In [ ]:
cli.main(["convert", 
        "--trial-file", trialFileOutputName, 
        "--search-path", pyramidSearchPath, 
        "--experiment", convertSpecs, 
        "--readers", 
        "ttl_reader.session_dir="+dataSearchPath+sessDir,
        "message_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_x_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_y_reader.session_dir="+dataSearchPath+sessDir,
        "pupil_reader.session_dir="+dataSearchPath+sessDir,
        "phy_reader.params_file="+sorted_out+"/phy/params.py"])

In [ ]:
sw.plot_sorting_summary(sorting_analyzer=sorter.cured_sorting_analyzer, curation=False, backend='spikeinterface_gui')